In [1]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [4]:
import matplotlib.pyplot as plt
import h5py
import rotation_mio as rot
import numpy as np
import barstrength2 as strng
import time_conversion as tiempo


h=0.732
G = 4.299e-6
a0=1
H0 = h*100
omega_lambda=0.716
omega_matter=0.1277/(h**2.)
omega0 = omega_lambda + omega_matter
vector2 = ('M31','MW','M33')
vector = ('00','01','03')


#path = 'home/ornela/SimCLUES/'
path = '/home/omarioni/'

atime = np.loadtxt(path + 'redshift_outputs.txt')
aexp = atime[:,2]
aexp = aexp[::-1]

path2 = '/mnt/is2/alejandro/ornella/'

snap = h5py.File(path2 + 'outputs_1/snap_496.h5py', 'r')
        
cm   = snap['subhalo_001/Center'].value
r200 = snap['subhalo_001/R200'].value

pstr = snap['subhalo_001/Str/Coordinates'].value
mstr = snap['subhalo_001/Str/Masses'].value
vstr  = snap['subhalo_001/Str/Velocities'].value

pdrk = snap['subhalo_001/Drk/Coordinates'].value
mdrk = snap['subhalo_001/Drk/Masses'].value
vdrk  = snap['subhalo_001/Drk/Velocities'].value

pgas = snap['subhalo_001/Gas/Coordinates'].value
mgas = snap['subhalo_001/Gas/Masses'].value
vgas  = snap['subhalo_001/Gas/Velocities'].value

z = a0/aexp[0] - 1.
Ht = H0*np.sqrt(omega_lambda+(1-omega0)*(1+z)**2+omega_matter*(1+z)**3)

#---aca paso las coordenadas respecto al centro de la galaxia------
xstr = (pstr[:,0]-cm[0])*aexp[0]/h
ystr = (pstr[:,1]-cm[1])*aexp[0]/h
zstr = (pstr[:,2]-cm[2])*aexp[0]/h
rstr = np.sqrt(xstr**2+ystr**2+zstr**2)

xdrk = (pdrk[:,0]-cm[0])*aexp[0]/h
ydrk = (pdrk[:,1]-cm[1])*aexp[0]/h
zdrk = (pdrk[:,2]-cm[2])*aexp[0]/h
rdrk = np.sqrt(xdrk**2+ydrk**2+zdrk**2)

xgas = (pgas[:,0]-cm[0])*aexp[0]/h
ygas = (pgas[:,1]-cm[1])*aexp[0]/h
zgas = (pgas[:,2]-cm[2])*aexp[0]/h
rgas = np.sqrt(xgas**2+ygas**2+zgas**2)

vx_str = vstr[:,0] *np.sqrt(aexp[0]) + Ht *xstr/1000.
vy_str = vstr[:,1] *np.sqrt(aexp[0]) + Ht *ystr/1000.
vz_str = vstr[:,2] *np.sqrt(aexp[0]) + Ht *zstr/1000.

vx_drk = vdrk[:,0] *np.sqrt(aexp[0]) + Ht *xdrk/1000.
vy_drk = vdrk[:,1] *np.sqrt(aexp[0]) + Ht *ydrk/1000.
vz_drk = vdrk[:,2] *np.sqrt(aexp[0]) + Ht *zdrk/1000.

vx_gas = vgas[:,0] *np.sqrt(aexp[0]) + Ht *xgas/1000.
vy_gas = vgas[:,1] *np.sqrt(aexp[0]) + Ht *ygas/1000.
vz_gas = vgas[:,2] *np.sqrt(aexp[0]) + Ht *zgas/1000.

#----------------------masas----------------------------
mstr = mstr/h
mdrk = mdrk/h
mgas = mgas/h

r200 = r200*aexp[0]/h
rgal=0.15*r200

# print aexp[0]
# print rgal 

limit = np.where(rstr<rgal)
r_sort = np.sort(rstr[limit])
r_indice = np.argsort(rstr[limit])

Mc_str = cumsum((mstr[limit])[r_indice])
M_gal = Mc_str[-1]

#------------------ calculamos M90------------------------------------------
razon = Mc_str/M_gal
cincuenta, = np.where(razon < 0.5)

r50 = r_sort[cincuenta][-1]             

#--------------------------------------------         
veloc,=np.where(rstr<r50)

#----------componentes de la velocidad del centro de masa------------
vxcm = sum(mstr[veloc]*vx_str[veloc])/sum(mstr[veloc])
vycm = sum(mstr[veloc]*vy_str[veloc])/sum(mstr[veloc])
vzcm = sum(mstr[veloc]*vz_str[veloc])/sum(mstr[veloc])

#----- velocidades de las estrellas respecto del centro de masa de la galaxia---------
vstr_x = vx_str - vxcm
vstr_y = vy_str - vycm
vstr_z = vz_str - vzcm

vdrk_x = vx_drk - vxcm
vdrk_y = vy_drk - vycm
vdrk_z = vz_drk - vzcm

vgas_x = vx_gas - vxcm
vgas_y = vy_gas - vycm
vgas_z = vz_gas - vzcm

e1x,e2x,e3x,e1y,e2y,e3y,e1z,e2z,e3z = rot.rot1(mstr,xstr,ystr,zstr,vstr_x,vstr_y,vstr_z,3*aexp[0])

##posiciones de partículas que se quiere graficar
##como lo de arriba me da los versores hago las posiciones con esto 

xn_str = e1x*xstr + e1y*ystr + e1z*zstr
yn_str = e2x*xstr + e2y*ystr + e2z*zstr
zn_str = e3x*xstr + e3y*ystr + e3z*zstr
vxn_str = e1x*vstr_x + e1y*vstr_y + e1z*vstr_z
vyn_str = e2x*vstr_x + e2y*vstr_y + e2z*vstr_z
vzn_str = e3x*vstr_x + e3y*vstr_y + e3z*vstr_z

rn_str = np.sqrt(xn_str**2+yn_str**2+zn_str**2)

xn_drk = e1x*xdrk + e1y*ydrk + e1z*zdrk
yn_drk = e2x*xdrk + e2y*ydrk + e2z*zdrk
zn_drk = e3x*xdrk + e3y*ydrk + e3z*zdrk
vxn_drk = e1x*vdrk_x + e1y*vdrk_y + e1z*vdrk_z
vyn_drk = e2x*vdrk_x + e2y*vdrk_y + e2z*vdrk_z
vzn_drk = e3x*vdrk_x + e3y*vdrk_y + e3z*vdrk_z

rn_drk = np.sqrt(xn_drk**2+yn_drk**2+zn_drk**2)

xn_gas = e1x*xgas + e1y*ygas + e1z*zgas
yn_gas = e2x*xgas + e2y*ygas + e2z*zgas
zn_gas = e3x*xgas + e3y*ygas + e3z*zgas
vxn_gas = e1x*vgas_x + e1y*vgas_y + e1z*vgas_z
vyn_gas = e2x*vgas_x + e2y*vgas_y + e2z*vgas_z
vzn_gas = e3x*vgas_x + e3y*vgas_y + e3z*vgas_z

rn_gas = np.sqrt(xn_gas**2+yn_gas**2+zn_gas**2)

limit_str = np.where(rn_str<rgal)
limit_drk = np.where(rn_drk<rgal)
limit_gas = np.where(rn_gas<rgal)

stars =np.ndarray([np.size(xn_str[limit_str]),7])
stars[:,0]=mstr[limit_str]*1e10
stars[:,1]=xn_str[limit_str]
stars[:,2]=yn_str[limit_str]
stars[:,3]=zn_str[limit_str]
stars[:,4]=vxn_str[limit_str]
stars[:,5]=vyn_str[limit_str]
stars[:,6]=vzn_str[limit_str]

# print xn_str[limit_str]
darkm =np.ndarray([np.size(xn_drk[limit_drk]),7])
darkm[:,0]=mdrk[limit_drk]*1e10
darkm[:,1]=xn_drk[limit_drk]
darkm[:,2]=yn_drk[limit_drk]
darkm[:,3]=zn_drk[limit_drk]
darkm[:,4]=vxn_drk[limit_drk]
darkm[:,5]=vyn_drk[limit_drk]
darkm[:,6]=vzn_drk[limit_drk]

gas =np.ndarray([np.size(xn_gas[limit_gas]),7])
gas[:,0]=mgas[limit_gas]*1e10
gas[:,1]=xn_gas[limit_gas]
gas[:,2]=yn_gas[limit_gas]
gas[:,3]=zn_gas[limit_gas]
gas[:,4]=vxn_gas[limit_gas]
gas[:,5]=vyn_gas[limit_gas]
gas[:,6]=vzn_gas[limit_gas]

np.savetxt('/home/omarioni/Barras_GdGs/Barras_Gd/_data/C_str.dat', stars, fmt='%12.6f')
np.savetxt('/home/omarioni/Barras_GdGs/Barras_Gd/_data/C_drk.dat', darkm, fmt='%12.6f')
np.savetxt('/home/omarioni/Barras_GdGs/Barras_Gd/_data/C_gas.dat', gas, fmt='%12.6f')

In [5]:
import matplotlib.pyplot as plt
import h5py
import rotation_mio as rot
import numpy as np
import barstrength2 as strng
import sphviewer as sph

a0 = 1.
vector = (1,2,4)
vector2 = ('M31', 'MW')
vector3 = ('A','B')
carpeta = ('9in1_M31/', '9in1_MW/')

#path = 'home/ornela/SimCLUES/'
path = '/mnt/sersic2/omarioni/'


snap = h5py.File(path + 'Gasoline/outputs2/snap_1024.h5py', 'r')

cm   = snap['subhalo_002/Center'].value
r200 = snap['subhalo_002/R200'].value
time = snap['subhalo_002/Time'].value
h    = snap['subhalo_002/h'].value
aexp = snap['subhalo_002/aexp'].value

pstr = snap['subhalo_002/Str/Coordinates'].value
mstr = snap['subhalo_002/Str/Masses'].value
vstr  = snap['subhalo_002/Str/Velocities'].value

pdrk = snap['subhalo_002/Drk/Coordinates'].value
mdrk = snap['subhalo_002/Drk/Masses'].value
vdrk = snap['subhalo_002/Drk/Velocities'].value

pgas = snap['subhalo_002/Gas/Coordinates'].value
mgas = snap['subhalo_002/Gas/Masses'].value
vgas = snap['subhalo_002/Gas/Velocities'].value

z = a0/aexp - 1.
#---aca paso las coordenadas respecto al centro de la galaxia------
xstr = pstr[:,0]-cm[0]
ystr = pstr[:,1]-cm[1]
zstr = pstr[:,2]-cm[2]
rstr = np.sqrt(xstr**2+ystr**2+zstr**2)

xdrk = pdrk[:,0]-cm[0]
ydrk = pdrk[:,1]-cm[1]
zdrk = pdrk[:,2]-cm[2]
rdrk = np.sqrt(xdrk**2+ydrk**2+zdrk**2)

xgas = pgas[:,0]-cm[0]
ygas = pgas[:,1]-cm[1]
zgas = pgas[:,2]-cm[2]
rgas = np.sqrt(xgas**2+ygas**2+zgas**2)

vx_str = vstr[:,0] 
vy_str = vstr[:,1] 
vz_str = vstr[:,2] 

vx_drk = vdrk[:,0] 
vy_drk = vdrk[:,1] 
vz_drk = vdrk[:,2] 

vx_gas = vgas[:,0] 
vy_gas = vgas[:,1] 
vz_gas = vgas[:,2] 

#----------------------masas----------------------------

r200 = r200*aexp
rgal=0.15*r200

limit = np.where(rstr<rgal)
r_sort = np.sort(rstr[limit])
r_indice = np.argsort(rstr[limit])

Mc_str = cumsum((mstr[limit])[r_indice])
M_gal = Mc_str[-1]

#------------------ calculamos M90------------------------------------------
razon = Mc_str/M_gal
cincuenta, = np.where(razon < 0.5)

r50 = r_sort[cincuenta][-1]             

#--------------------------------------------         
veloc,=np.where(rstr<r50)

#----------componentes de la velocidad del centro de masa------------
vxcm = sum(mstr[veloc]*vx_str[veloc])/sum(mstr[veloc])
vycm = sum(mstr[veloc]*vy_str[veloc])/sum(mstr[veloc])
vzcm = sum(mstr[veloc]*vz_str[veloc])/sum(mstr[veloc])

#----- velocidades de las estrellas respecto del centro de masa de la galaxia---------
vstr_x = vx_str - vxcm
vstr_y = vy_str - vycm
vstr_z = vz_str - vzcm

vdrk_x = vx_drk - vxcm
vdrk_y = vy_drk - vycm
vdrk_z = vz_drk - vzcm

vgas_x = vx_gas - vxcm
vgas_y = vy_gas - vycm
vgas_z = vz_gas - vzcm

e1x,e2x,e3x,e1y,e2y,e3y,e1z,e2z,e3z = rot.rot1(mstr,xstr,ystr,zstr,vstr_x,vstr_y,vstr_z,3*aexp)

##posiciones de partículas que se quiere graficar
##como lo de arriba me da los versores hago las posiciones con esto 

xn_str = e1x*xstr + e1y*ystr + e1z*zstr
yn_str = e2x*xstr + e2y*ystr + e2z*zstr
zn_str = e3x*xstr + e3y*ystr + e3z*zstr
vxn_str = e1x*vstr_x + e1y*vstr_y + e1z*vstr_z
vyn_str = e2x*vstr_x + e2y*vstr_y + e2z*vstr_z
vzn_str = e3x*vstr_x + e3y*vstr_y + e3z*vstr_z

xn_drk = e1x*xdrk + e1y*ydrk + e1z*zdrk
yn_drk = e2x*xdrk + e2y*ydrk + e2z*zdrk
zn_drk = e3x*xdrk + e3y*ydrk + e3z*zdrk
vxn_drk = e1x*vdrk_x + e1y*vdrk_y + e1z*vdrk_z
vyn_drk = e2x*vdrk_x + e2y*vdrk_y + e2z*vdrk_z
vzn_drk = e3x*vdrk_x + e3y*vdrk_y + e3z*vdrk_z

xn_gas = e1x*xgas + e1y*ygas + e1z*zgas
yn_gas = e2x*xgas + e2y*ygas + e2z*zgas
zn_gas = e3x*xgas + e3y*ygas + e3z*zgas
vxn_gas = e1x*vgas_x + e1y*vgas_y + e1z*vgas_z
vyn_gas = e2x*vgas_x + e2y*vgas_y + e2z*vgas_z
vzn_gas = e3x*vgas_x + e3y*vgas_y + e3z*vgas_z

limit_str = np.where(rstr<rgal)
limit_drk = np.where(rdrk<rgal)
limit_gas = np.where(rgas<rgal)

stars =np.ndarray([np.size(xn_str[limit_str]),7])
stars[:,0]=mstr[limit_str]
stars[:,1]=xn_str[limit_str]
stars[:,2]=yn_str[limit_str]
stars[:,3]=zn_str[limit_str]
stars[:,4]=vxn_str[limit_str]
stars[:,5]=vyn_str[limit_str]
stars[:,6]=vzn_str[limit_str]

darkm =np.ndarray([np.size(xn_drk[limit_drk]),7])
darkm[:,0]=mdrk[limit_drk]
darkm[:,1]=xn_drk[limit_drk]
darkm[:,2]=yn_drk[limit_drk]
darkm[:,3]=zn_drk[limit_drk]
darkm[:,4]=vxn_drk[limit_drk]
darkm[:,5]=vyn_drk[limit_drk]
darkm[:,6]=vzn_drk[limit_drk]

gas =np.ndarray([np.size(xn_gas[limit_gas]),7])
gas[:,0]=mgas[limit_gas]
gas[:,1]=xn_gas[limit_gas]
gas[:,2]=yn_gas[limit_gas]
gas[:,3]=zn_gas[limit_gas]
gas[:,4]=vxn_gas[limit_gas]
gas[:,5]=vyn_gas[limit_gas]
gas[:,6]=vzn_gas[limit_gas]

np.savetxt('/home/omarioni/Barras_GdGs/Barras_Gs/_data/D_str.dat', stars, fmt='%12.6f')
np.savetxt('/home/omarioni/Barras_GdGs/Barras_Gs/_data/D_drk.dat', darkm, fmt='%12.6f')
np.savetxt('/home/omarioni/Barras_GdGs/Barras_Gs/_data/D_gas.dat', gas, fmt='%12.6f')